### Basic of SQL with PySpacks

In this notebook we are going to learn the sql basics using pyspacks.

First we are going to start by installing all required packages.

In [18]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# install findspark using pip
!pip install -q findspark

The after that we are going to initialize the sparks context.

In [19]:


# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In the following code cell we are going to import all the packages that we will be using in this notebook.

In [29]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf


We are going to load the dataset that was downloaded from kaggle.

https://www.kaggle.com/c/titanic/data

We only care about the `train.csv` file.


In the following code cell we are then going to load our data using sparks.

In [20]:
dataframe = spark.read.csv("train.csv", header=True, inferSchema=True)
dataframe

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,null,S
6,0,3,"Moran, Mr. James",male,null,0,0,330877,8.4583,null,Q
7,0,1,"McCarthy, Mr. Tim...",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. ...",male,2.0,3,1,349909,21.075,null,S
9,1,3,"Johnson, Mrs. Osc...",female,27.0,0,2,347742,11.1333,null,S
10,1,2,"Nasser, Mrs. Nich...",female,14.0,1,0,237736,30.0708,null,C


We can check the schema by using the method called `printSchema`

In [21]:
dataframe.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



We can limit the number of rows we want by using the `limit` method. Here is an example that takes the first 3 rows in the dataframe.

In [22]:
dataframe.limit(3)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,null,S


We can use the method called `select` to select the column names that we are intrested in for example.

In [23]:
dataframe.select("PassengerId", "Survived", "Pclass").limit(3)

PassengerId,Survived,Pclass
1,0,3
2,1,1
3,1,3


We can use the `where` method to filter rows in the data based on their column values. Here is an example.

In [25]:
dataframe.where((dataframe.Age > 25) & (dataframe.Sex == 'male')).select("Age", "Sex", "Name", "Survived").limit(3)

Age,Sex,Name,Survived
35.0,male,"Allen, Mr. Willia...",0
54.0,male,"McCarthy, Mr. Tim...",0
39.0,male,"Andersson, Mr. An...",0


We can do data aggregation on our dataframe for example if we want to calculate the average based on the `Fare` column we can do it as follows:

In [26]:
dataframe.agg({"Fare": "avg"})

avg(Fare)
32.2042079685746


We can also group items and do aggregaration for example.

In [27]:
dataframe.groupBy('Pclass').agg({'Fare':'avg'}).orderBy('Pclass', ascending=False)

Pclass,avg(Fare)
3,13.675550101832997
2,20.66218315217391
1,84.15468749999992


We can use the filter method to filter rows in the dataframe. Here is an example:

In [28]:
dataframe.filter(dataframe.Age > 25).agg({'Fare':'avg'})

avg(Fare)
37.61960169491524


We can apply a function to a dataframe column just like in pandas, however this function need to be a `udf`. We are going to create a function called `round_float` that will round the `Fare` column to become an integer. This can be done as follows:

In [30]:
def round_float(x):
  return int(x)

round_float_udf = udf(round_float, IntegerType())

dataframe.select('PassengerId', 'Fare', round_float_udf('Fare').alias('Fare Rounded')).limit(3)

PassengerId,Fare,Fare Rounded
1,7.25,7
2,71.2833,71
3,7.925,7


We can create our dataframe as a table by using the `createOrReplaceTempView` method. Let's create a table called `titanic` in our dataset that we will use to query using sql.

In [31]:
dataframe.createOrReplaceTempView("titanic")

We can use the `spack.sql` method to execute queries based on this table called `titanic` that we just created. Here is how it can be done.

In [32]:
spark.sql("SELECT `Fare`, `Age`, `PassengerId` FROM `titanic` WHERE Age > 25").limit(3)

Fare,Age,PassengerId
71.2833,38.0,2
7.925,26.0,3
53.1,35.0,4
